## 調整textGCN句子結構
+ read
+ `../result_MATE/{args_data}.train_out`

In [1]:
from nltk.parse import CoreNLPParser
from stanza.server import CoreNLPClient
import stanza
import os
import re
import pandas as pd
from tqdm import tqdm
import time

In [34]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
# stanza.install_corenlp()
# stanza.download('en')
# stanza_nlp = stanza.Pipeline('en')
# os.environ['CORENLP_HOME'] = "./stanza_corenlp"

In [35]:
args_domain = "REST"
args_data = "yelp"
args_kr = "DP+_2016" # "DP_2016", "DP+_2016" # type of key_relation
args_aspect = 5

## 定義重要的dependency type

+ DP_2016 的 prep 換成 case ; xsubj 目前找不到對應的
    + prep 換成 case;
    + xsubj 目前找不到對應的 

+ DP+_2016 
    + prep 換成 case;
    + acomp => xcomp ; 
    + xsubj 目前找不到對應的 
    + pobj => nmod (nmod範圍比較廣)
    + poss => 文件說是 case 的一種 (需要確認 nmod:poss 和他的關係)

In [36]:
asp_dict = {
        "REST": ["AMB", "DRI", "FOD", "LOC", "SRV"],
        "LAPTOP": ["SUPPORT", "OS", "DISPLAY", "BATTERY", "COMPANY", "MOUSE", "SOFTWARE", "KEYBOARD"]
    }

relation_dict = {
    "DP_2016": ["amod", "case", "nsubj", "csubj", "xsubj", "dobj", "iobj", "conj"], 
    "DP+_2016": ["amod", "case", "nsubj", "csubj", "xsubj", "dobj", "iobj", "conj", 
            "advmod", "dep", "cop", "mark", "nsubjpass", "pobj", "acomp", "xcomp", "csubjpass", "poss"],
}


asp_name_list = asp_dict[args_domain] # domain's aspect name
asp_name_list

['AMB', 'DRI', 'FOD', 'LOC', 'SRV']

In [5]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')

2022-06-20 05:51:15 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-06-20 05:51:15 INFO: Use device: gpu
2022-06-20 05:51:15 INFO: Loading: tokenize
2022-06-20 05:51:19 INFO: Loading: pos
2022-06-20 05:51:19 INFO: Loading: lemma
2022-06-20 05:51:19 INFO: Loading: depparse
2022-06-20 05:51:20 INFO: Done loading processors!


In [6]:
# # for test(看nlp有沒有正常啟動)
text = 'A blog post using Stanford CoreNLP Server in New York.I like great pizza here'
doc = nlp(text)
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

id: 1	word: A	head id: 3	head: post	deprel: det
id: 2	word: blog	head id: 3	head: post	deprel: compound
id: 3	word: post	head id: 0	head: root	deprel: root
id: 4	word: using	head id: 3	head: post	deprel: acl
id: 5	word: Stanford	head id: 7	head: Server	deprel: compound
id: 6	word: CoreNLP	head id: 7	head: Server	deprel: compound
id: 7	word: Server	head id: 4	head: using	deprel: obj
id: 8	word: in	head id: 9	head: New	deprel: case
id: 9	word: New	head id: 7	head: Server	deprel: nmod
id: 10	word: York	head id: 9	head: New	deprel: flat
id: 11	word: .	head id: 3	head: post	deprel: punct
id: 1	word: I	head id: 2	head: like	deprel: nsubj
id: 2	word: like	head id: 0	head: root	deprel: root
id: 3	word: great	head id: 4	head: pizza	deprel: amod
id: 4	word: pizza	head id: 2	head: like	deprel: obj
id: 5	word: here	head id: 2	head: like	deprel: advmod


In [ ]:
# # for test
# dl = []
# text = 'A blog post using Stanford CoreNLP Server in New York.I like great pizza here'
# doc = nlp(text)

# i = 0 # sentence id
# for sent in doc.sentences:
#     for word in sent.words:
#         dr = [i,word.id,word.text,word.head,sent.words[word.head-1].text if word.head > 0 else "root",word.deprel]
#         dl.append(dr)
#     i = i + 1
    
# test_df = pd.DataFrame(data=dl, columns=['sent_id','word_id','word','head_id','head_word','type'])
# test_df

## 分析原本的文集

+ 先分析原本的文集，儲存成資料框

In [24]:
# 開啟 MATE 訓練資料分類 (yelp.train_out)
result_pair = []
f = open(f"../../model_result/JASA_trainout/R0.train_out", 'r')
for line in f:
    spline = line.split('\t')
    res_list = list(map(float, spline[1:len(asp_name_list)+1]))
    result_pair.append({"scode": int(spline[0][2:-1]), 
                        "aspect": res_list.index(max(res_list)), 
                        "text":spline[-1].strip()[2:-1]})
f.close()

In [25]:
# 資料前處理
df = pd.DataFrame(result_pair) # 49994
df['text'] = df['text'].str.replace('^b\'','')
df['text'] = df['text'].str.replace('^b"','')
df['text'] = df['text'].str.replace(r'\\n\'','', regex=True) 
df['text'] = df['text'].str.replace(r'\\n"','', regex=True) 

In [ ]:
# parse sentences
dl = []
for index, row in tqdm(df.iterrows()):    
    # parse sentence
    doc = nlp(row['text'])     
    i = 0 # sentence id
    for sent in doc.sentences:
        for word in sent.words:
            dr = [row['scode'],i,
                  word.id,word.text, word.head, 
                  sent.words[word.head-1].text if word.head > 0 else "root", word.deprel]
            dl.append(dr)
        i = i + 1

44041it [45:27,  9.23it/s]

In [ ]:
# # list to dataframe
dependency_df = pd.DataFrame(data=dl, columns=['doc_id','sent_id','word_id','word','head_id','head_word','type'])
# dependency_df

In [ ]:
# # save as dataframe
# dependency_df.to_csv(r'dependency_df.csv', index = False)

In [ ]:
# dependency_df.shape # (5648250, 7)

## 匯入解析完成的句子資料

In [37]:
dependency = pd.read_csv('dependency_df.csv')
dependency.shape

(4713361, 7)

In [31]:
dependency.type.unique()

array(['root', 'det', 'obj', 'cc', 'conj', 'case', 'obl', 'nmod:poss',
       'nsubj', 'acl:relcl', 'cop', 'advmod', 'parataxis', 'aux', 'ccomp',
       'mark', 'xcomp', 'advcl', 'compound:prt', 'compound', 'amod',
       'nmod', 'expl', 'obl:tmod', 'aux:pass', 'csubj', 'nsubj:pass',
       'discourse', 'fixed', 'flat', 'det:predet', 'obl:npmod',
       'cc:preconj', 'acl', 'appos', 'iobj', 'punct', 'nmod:tmod',
       'nummod', 'list', 'dislocated', 'nmod:npmod', 'goeswith',
       'vocative', 'reparandum', 'flat:foreign', 'csubj:pass', 'dep',
       'orphan', '<PAD>'], dtype=object)

In [42]:
len(dependency.doc_id.unique())

48620

In [38]:
dependency.head(1)

,doc_id,sent_id,word_id,word,head_id,head_word,type
0,4729,0,1,had,0,root,root


In [44]:
dependency[dependency['doc_id']==1].word.unique()

array(['yet', 'another', 'five', 'star', 'review', 'for', 'this',
       'fantastic', 'place', 'i', 'joked', 'around', 'with', 'the',
       'owner', 'about', 'giving', 'him', 'a', 'terrible', 'so', 'he',
       'could', 'get', 'little', 'bit', 'of', 'respite', 'from', 'all',
       'yelpers', 'coming', 'in', 'there', 'but', 'was', 'some', 'pizza',
       'sure', 'not', 'only', 'great', 'it', 'run', 'by', 'really',
       'people', 'we', 'had', 'wonderful', 'conversation', 'proprietors',
       'while', 'my', 'partner', 'and', 'devoured', 'most', 'an',
       'entire', 'pie', 'to', 'ourselves', 'definitely', 'looking',
       'forward', 'next', 'time', 'go'], dtype=object)

## 方法一：解析並過濾句子
字詞需要有和重要的dependency type 連線，才能留下來
+ text：輸入的sentence
+ relation_type：重要的dependency type

In [25]:
# input sentence
# output adjust sentence
def filter_sentence(text, relation_type, quiet=True):
    key_relation = relation_dict[relation_type]
    doc = nlp(text)
    # for sent in doc.sentences 
    doc_list = [] # list for sentence
    for sent in doc.sentences:
        words = [] # all words
        de_dict = {} # relation pair id
        de_dict_word = {} # relation pair text

        for word in sent.words:
            key = word.deprel # dependency type        
            head_id = word.head 
            node_id = word.id
            head = sent.words[word.head-1].text if word.head > 0 else "root" # head of node
            node = word.text # node word
            words.append(node) # sentence

            if key in key_relation:   
                if quiet == False:
                    print(key,":",node_id,node,head_id,head)
                de_dict.setdefault(key, []).append(head_id)
                de_dict.setdefault(key, []).append(node_id)

                de_dict_word.setdefault(key, []).append(head)
                de_dict_word.setdefault(key, []).append(node)

        keep = list({x for v in de_dict.values() for x in v}) # word id to keep
        keep_word = list({x for v in de_dict_word.values() for x in v}) # word text to keep
        s = [words[i-1] for i in keep] # filtered sentence
        doc_list.append(s)

    sentence = [item for sublist in doc_list for item in sublist]
    sentence = ' '.join(sentence)
    return sentence

## 讀進原本的資料，轉換成新的train_sumout

In [33]:
start = time.time()
train_sumout = "" # adjust sentence
f = open(f"../result_MATE/{args_data}.train_out", 'r')
print(f"read ../result_MATE/{args_data}.train_out")

for line in f:
    sp = line.split('\t')
    
    # get sentence scode
    scode = int(sp[0][2:-1])
    # replace string in sentence
    text = sp[len(asp_name_list)+1].strip()
    text = re.sub('^b\'','',text)
    text = re.sub('^b"','',text)
    text = text.replace("\\n'", "")
    text = text.replace("\\n\"", "")
    
    # adjust sentence
    text = filter_sentence(text, args_kr) 
    
    # adjust sumout
    train_sumout += 'b"' + str(scode) + '"'
    train_sumout += '\t' + 'b"' + str(text) + '"' + '\n' 
    
f.close()
end = time.time()
print("time cost",end-start)

read ../result_MATE/yelp.train_out
time cost 3659.9488260746


In [34]:
# save as new train_out
savef = '../result_ASSA/{0}.train_out'.format(args_kr) # args_kr "test"
fsum = open(savef, 'w') # args.trainout_name
print('save as {}'.format(savef))
fsum.write(train_sumout)
fsum.close()

save as ../result_ASSA/DP+_2016.train_out


## 方法二：過濾句子的function（有解析資料框）
字詞需要有和重要的dependency type 連線，才能留下來
+ text：解析完的df
+ relation_type：重要的dependency type

In [ ]:
dependency.head(3)

In [ ]:
# 確認一下：這幾個不在corpus裡面 {'acomp', 'csubjpass', 'dobj', 'nsubjpass', 'pobj', 'poss', 'xsubj'}

In [ ]:
# # 可以維持順序的方法（先不用）
# s = []
# doc_id = None
# all_s = []
# for index, row in dependency.iterrows():    
#     temp_id = row['doc_id']    
#     if temp_id != doc_id:
#         all_s.append([doc_id,s])
#         doc_id = temp_id
#         s = []
#     else:        
#         if row['type'] in sel_relation:
#             s.append(row['word'])

In [10]:
sel_relation = relation_dict[args_kr] # DP_2016 args_kr
filter_df = dependency[dependency['type'].isin(sel_relation)]
filter_df = filter_df.groupby('doc_id')['word'].apply(list).reset_index(name='sentence')

train_sumout = ""
for index, row in filter_df.iterrows():    
    scode = row['doc_id']    
    text = [x for x in row['sentence'] if str(x) != 'nan']
    text = ' '.join(text)
    # adjust sumout
    train_sumout += 'b"' + str(scode) + '"'
    train_sumout += '\t' + 'b"' + str(text) + '"' + '\n' 

In [12]:
# save as new train_out
savef = '../result_ASSA/{0}.train_out'.format(args_kr) # args_kr
fsum = open(savef, 'w') # args.trainout_name
print('save as {}'.format(savef))
fsum.write(train_sumout)
fsum.close()

save as ../result_ASSA/DP+_2016.train_out


## 檢查：儲存的格式對不對(調整後的檔案)
+ 讀取的檔案只有scode和調整後的句子
+ rest 的訓練資料應該要有 49994 筆

In [18]:
result_pair = []
# openf = "../result_ASSA/rest_R0.train_out" # 原始句子檔案 
openf = "../result_ASSA/DP+_2016.train_out" # 原始句子檔案 
# openf = f"../result_ASSA/{args_kr}.train_out" # args_kr "DP_2016", "DP+_2016"
# 開啟 MATE 訓練資料分類 (yelp.train_out)
f = open(openf, 'r')
for line in f:
    sp = line.split('\t')
    result_pair.append({"scode": int(sp[0][2:-1]), "text":sp[-1].strip()})
f.close()

In [24]:
len(result_pair)
# result_pair[7327]

49974

## 讀取原始格式的檔案

In [16]:
# 驗證儲存的格式對不對(原檔)
result_pair = []
# 開啟 MATE 訓練資料分類 (yelp.train_out)
f = open(f"../result_MATE/yelp.train_out", 'r')
for line in f:
    sp = line.split('\t')
    res_list = list(map(float, sp[1:len(asp_name_list)+1]))
    result_pair.append({"scode": int(sp[0][2:-1]), "aspect": res_list.index(max(res_list)), "text":sp[len(asp_name_list)+1].strip()})
f.close()

In [ ]:
df = pd.DataFrame(result_pair)

In [ ]:
df.head(3)

In [ ]:
# # 查看含有某個關鍵字的句子
# keyword_list = ['food']
# select_aspect = 1
# df[
#     (df['text'].str.contains('|'.join(keyword_list),regex=True, case=False)) &
#     (df['aspect'] == select_aspect)]

拆解單句結構
+ 看node,edge結構
+ 看句型樹狀圖

In [ ]:
# # 樹的結構圖
# parser = CoreNLPParser('http://localhost:9008') # 使用開在loclahost:9008的coreNLP服務
# text = 'A blog post using Stanford CoreNLP Server. Visit www.khalidalnajjar.com for more details.'
# parser.raw_parse(text)

# fox_parsed = next(parser.raw_parse(text))
# fox_parsed.pretty_print()


# child 和 rdge 的結構
# text = 'I didn\'t tell her. Do you know her name?'
# document = client.annotate(text)

# # 示範句法結構
# for i, sent in enumerate(document.sentence):
    
#     # get the constituency parse of the sentence
#     constituency_parse = sent.parseTree # child 
#     print(constituency_parse)

#     # get the dependency parse of the first sentence
#     dependency_parse = sent.basicDependencies # edge
#     print(dependency_parse)
#     break # 只顯示第一句!!

## related links
minipar dependency list
+ https://gate.ac.uk/releases/gate-7.0-build4195-ALL/doc/tao/splitch17.html
+ GateNLP github: https://github.com/GateNLP/userguide/blob/master/parsers.tex
    (\subsect[sec:parsers:minipar:GR]{Grammatical Relationships})
    
Stanford 
+ UD v1 （目前是用第一版的） 
    + http://universaldependencies.org/docsv1/en/dep/index.html
    + from http://universaldependencies.org/docsv1/
        (> english > english relation)
+ different version compare (比對Stanford新舊版本的名字)
    + https://nlp.stanford.edu/pubs/USD_LREC14_paper_camera_ready.pdf

+ old version of stanford
    + https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf
    + 有舊版每個類別的解釋

+ 官網說明：
    + https://nlp.stanford.edu/software/stanford-dependencies.shtml
    + Since version 3.5.2 the Stanford Parser and Stanford CoreNLP output grammatical relations in the Universal Dependencies v1 representation by default.

dependency type mapping
+ stackoverflow
+ 相似度的研究用：amod, prep, nsubj, csubj, xsubj, dobj and iobj (and conj)
+ https://linguistics.stackexchange.com/questions/19872/mapping-minipar-dependencies-to-stanford-parser-dependencies
+ 相似的paper 
+ http://www.mitpressjournals.org/doi/pdfplus/10.1162/coli_a_00034

related link
+ 官網示範
+ https://corenlp.run/
+ UD v2 https://universaldependencies.org/en/dep/index.html
+ from https://universaldependencies.org/
    (> English > documentation page > Syntax > English relations)

source link
nlp pipeline
+ https://stanfordnlp.github.io/stanza/depparse.html

dependency list
+ stackoverflow: https://stackoverflow.com/questions/10687173/stanford-dependencies-list
+ paper-Stanford typed dependencies manua(2008): https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf